In [ ]:
# Install relavent packages
!pip install scikit-learn
!pip install scikeras
!pip install tensorflow

In [ ]:
# import realavent packages
import pandas as pd
import math
import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
# label categories, load data
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
df_diabetes = pd.read_csv('/content/diabetes.csv', names = names)

df_diabetes.head() #View data


,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# seprate x and y data by key
X = df_diabetes[['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age']]
y = df_diabetes['class']

In [ ]:
# rescale data to number between zero and one
scaler = MinMaxScaler(feature_range=(0, 1))
X_rescaled = scaler.fit_transform(X)
X = pd.DataFrame(data = X_rescaled, columns = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'])
X.head(3)

,preg,plas,pres,skin,test,mass,pedi,age
0,0.352941,0.743719,0.590164,0.353535,0.0,0.500745,0.234415,0.483333
1,0.058824,0.427136,0.540984,0.292929,0.0,0.396423,0.116567,0.166667
2,0.470588,0.919598,0.524590,0.000000,0.0,0.347243,0.253629,0.183333


In [ ]:
# convert to numpy array
X= X.to_numpy()
y = y.to_numpy()

In [ ]:
# seperate training data and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75, random_state = 1, stratify =y, shuffle = True)

In [ ]:
# create variable node function
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    # initalize layers array
    layers = []

    # define node increment from first to last layer: negative number
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes

    # iteratively add number of nodes in each layer to list
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment

    return layers



In [ ]:
def createmodel(n_layers, first_layer_nodes, last_layer_nodes, activation_func, loss_func):
    # initialize model, define nodes using linear node function
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    # add layers to model
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))

    # add single node as the final layer for binary classification
    model.add(Dense(1, activation=activation_func))
    model.compile(optimizer='adam', loss=loss_func, metrics = ["accuracy"])

    return model

# define set of hyperparameters
activation_funcs = ['sigmoid', 'relu', 'tanh']
first_layer_nodes = (64,32,16)
last_layer_nodes = (4)
activation_funcs = ['sigmoid', 'relu', 'tanh']
loss_funcs = ['binary_crossentropy','hinge']
n_layers=(2,3)
epochs = [30,60]

#Wrap model into scikit-learn
model =  KerasClassifier(build_fn=createmodel, verbose = False, activation_func = activation_funcs
                         , first_layer_nodes = first_layer_nodes, last_layer_nodes = last_layer_nodes,
                         loss_func=loss_funcs, n_layers=n_layers)


In [ ]:
# define grid of hyperparameters
p_grid = dict(n_layers=n_layers, first_layer_nodes = first_layer_nodes, last_layer_nodes = last_layer_nodes,
              activation_func = activation_funcs, loss_func = loss_funcs, batch_size = [100], epochs = epochs)

# create iterative model that trains and test data on every permutation of hyperparameters
grid = GridSearchCV(estimator = model, param_grid = p_grid)

In [ ]:
# pass data to model
grid.fit(X,y)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using 

GridSearchCV(estimator=KerasClassifier(activation_func=['sigmoid', 'relu', 'tanh'], build_fn=<function createmodel at 0x7d5f0d5ba0e0>, first_layer_nodes=(64, 32, 16), last_layer_nodes=4, loss_func=['binary_crossentropy', 'hinge'], n_layers=(2, 3), verbose=False),
             param_grid={'activation_func': ['sigmoid', 'relu', 'tanh'],
                         'batch_size': [100], 'epochs': [30, 60],
                         'first_layer_nodes': [64, 32, 16],
                         'last_layer_nodes': [4],
                         'loss_func': ['binary_crossentropy', 'hinge'],
                         'n_layers': [2, 3]})

In [ ]:
# display highest accuracy and asociated model parameters
print(f'accuracy = {grid.best_score_}')
optimal_params = grid.best_params_
for key in optimal_params:
  print(f"{key} = {optimal_params[key]} ")

accuracy = 0.7539852304558187
activation_func = relu 
batch_size = 100 
epochs = 30 
first_layer_nodes = 64 
last_layer_nodes = 4 
loss_func = binary_crossentropy 
n_layers = 3 


In [ ]:
# save grid of results as CSV
data_export = pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']].to_csv('GridOptimization.csv')
